In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# train, test 데이터 로드
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

1. train.csv / test.csv : 타이타닉 탑승자들 중 일부의 인적 정보와 생존 여부 데이터  
PassengerID : 탑승객 고유 아이디
Survival : 탑승객 생존 유무 (0: 사망, 1: 생존)  
Pclass : 등실의 등급  
Name : 이름  
Sex : 성별  
Age : 나이  
Sibsp : 함께 탐승한 형제자매, 아내, 남편의 수  
Parch : 함께 탐승한 부모, 자식의 수  
Ticket :티켓 번호  
Fare : 티켓의 요금  
Cabin : 객실번호  
Embarked : 배에 탑승한 항구 이름 ( C = Cherbourn, Q = Queenstown, S = Southampton)


In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
test_df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
train_df.shape

(891, 12)

In [7]:
test_df.shape

(418, 11)

In [8]:
train_df.isnull().sum()
#train_df.info()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
test_df.isnull().sum()
#test_df.info()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

#1. 데이터 전처리


1. Age : 호칭에 따른 평균값
2. Cabin : 최빈값
3. Embarked : 최빈값


In [10]:
# 'Mr', 'Miss', 'Mrs', 'Ms', 'Master', 'Dr', Child로 이름 분류 후, 각 그룹의 평균값으로 결측치 채우기 (train 데이터)
age_dict_train = {}

# Mr, Ms, Miss, Mrs로 이름 분류
for title in ['Mr', 'Miss', 'Mrs', 'Ms', 'Master', 'Dr']:
    # 해당 타이틀을 가진 사람들의 나이 평균값 계산
    age = train_df[train_df['Name'].str.contains(title)]['Age'].mean()
    # 딕셔너리에 저장
    age_dict_train[title] = int(round(age))

# 타이틀에 해당되지 않는 사람들은 Child로 분류
age_dict_train['Child'] = train_df[train_df['Name'].apply(lambda x: 'Mr' not in x and
                                                          'Miss' not in x and
                                                          'Mrs' not in x and
                                                          'Ms' not in x and
                                                          'Master' not in x and
                                                          'Dr' not in x)]['Age'].mean()

In [11]:
# 'Mr', 'Miss', 'Mrs', 'Ms', 'Master', 'Dr', Child로 이름 분류 후, 각 그룹의 평균값으로 결측치 채우기 (test 데이터)
age_dict_test = {}

# MR, MS, MISS, MRS로 이름 분류
for title in ['Mr', 'Miss', 'Mrs','Master', 'Dr']:
    # 해당 타이틀을 가진 사람들의 나이 평균값 계산
    age = test_df[test_df['Name'].str.contains(title)]['Age'].mean()
    # 딕셔너리에 저장
    age_dict_test[title] = int(round(age))

# test데이터의 Ms가 1명이므로 train데이터의 결과값을 가져와서 대체
test_df.loc[test_df['Name'].str.contains('Ms'), 'Age'] = age_dict_train['Ms']

# 타이틀에 해당되지 않는 사람들은 Child로 분류
age_dict_test['Child'] = test_df[test_df['Name'].apply(lambda x: 'Mr' not in x and
                                                       'Miss' not in x and
                                                       'Mrs' not in x and
                                                       'Ms' not in x and
                                                       'Master' not in x and
                                                       'Dr' not in x)]['Age'].mean()

In [12]:
for title, age in age_dict_train.items():
    # 위에 해당 타이틀을 가진 결측치 index 추출
    idx = train_df[train_df['Name'].str.contains(title) & train_df['Age'].isnull()].index

    # 해당 index의 결측치를 해당 타이틀의 나이 평균값으로 채우기
    train_df.loc[idx, 'Age'] = age

In [13]:
for title, age in age_dict_test.items():
    # 위에 해당 타이틀을 가진 결측치 index 추출
    idx = test_df[test_df['Name'].str.contains(title) & test_df['Age'].isnull()].index

    # 해당 index의 결측치를 해당 타이틀의 나이 평균값으로 채우기
    test_df.loc[idx, 'Age'] = age

In [14]:
# Cabin과 Embarked는 최빈값으로 채우기
train_df.fillna(train_df['Cabin'].mode()[0], inplace=True)
train_df.fillna(train_df['Embarked'].mode()[0], inplace=True)

In [15]:
# Fare은 평균값, Cabin은 최빈값으로 채우기
test_df.fillna(test_df['Fare'].mean(), inplace=True)
test_df.fillna(test_df['Cabin'].mode()[0], inplace=True)

In [16]:
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [17]:
test_df.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64